In [ ]:
!pip install pyPDF2
!pip install faiss-cpu
!pip install sentence_transformers
!pip install -U langchain-community
!pip install huggingface_hub

In [ ]:
import os

MYHFKEY = "key"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = MYHFKEY

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import os

In [ ]:
pipe = pipeline("text2text-generation", model="google/flan-t5-large", max_new_tokens=100)

template = """ {question}"""
prompt_template = PromptTemplate.from_template(template)
llm = HuggingFacePipeline(pipeline=pipe)
chain = prompt_template | llm
myQuery = "In how many movies benoit cayla played ?"
chain.invoke(myQuery)

In [ ]:
llm("What Benoit is doing now ?")

In [ ]:
!wget https://raw.githubusercontent.com/datacorner/les-tutos-datacorner.fr/master/datasources/fakeinfos.pdf

In [ ]:
newData = PdfReader("/content/fakeinfos.pdf")
allcontent = ""
for i, page in enumerate(newData.pages):
  content = page.extract_text()
  if content:
    allcontent += content

In [ ]:
print(allcontent)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200,
                                              chunk_overlap = 20,
                                              length_function = len,
                                               separators=["\n\n", "\n", ".", " ", ""])
finalAllcontent = text_splitter.split_text(allcontent)
finalAllcontent

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
docSearch = FAISS.from_texts(finalAllcontent, embeddings)
newChain = load_qa_chain(llm, chain_type="stuff")
docs = docSearch.similarity_search(myQuery)
newChain.run(input_documents=docs, question=myQuery)

In [ ]:
newChain.run(input_documents=docs, question="What Benoit is doing now ?")